In [1]:
import itertools
import numpy as np
from tqdm import tqdm
import timeit
from IPython.display import display, Latex, Math

# Quiver mutation

We represent a quiver as a "B-matrix". This has entries $b_{ij}$, the number of arrows from vertex $i$ to vertex $j$ in the quiver. For arrows pointing $i\rightarrow j$ we define $b_{ji}=-b_{ij}$.

Quivers have an operation called mutation, which can be applied at any vertex $k$, which we write as $\mu_k$. We call this new quiver $\tilde{Q}$ so this means $$\tilde{Q}:=\mu_k(Q)$$. We write the entries of the new quiver $\tilde{Q}$ as $\tilde{b}_{ij}$, and these are given by

$$\tilde{b}_{ij}=\begin{cases}
-b_{ij} & i=k \:\mathrm{or}\: j=k \\
b_{ij} + \mathrm{sgn}(b_{ik})[b_{ik}b_{kj}]_+ & \mathrm{otherwise}
\end{cases}
$$

where the maxplus $[x]_+$ is defined as 

$$[x]_+=\begin{cases}
x & x>0 \\
0 & \mathrm{otherwise}
\end{cases}
$$

In [2]:
def max_plus(num):
    if num <= 0:
        return 0
    return num

def mutate(quiver, vertex):
    dim = quiver.shape[0]
    result = np.zeros(shape=(dim, dim))
    for i in range(dim):
        for j in range(dim):
            if (i == vertex or j == vertex):
                result[i,j] = -quiver[i,j]
            else:
                result[i,j] = quiver[i,j]+np.sign(quiver[i,vertex])*max_plus(quiver[i,vertex]*quiver[vertex,j])
    return np.array(result)

An example quiver and mutation

In [3]:
quiver1 = np.array([
    [0, 1, -1, 0, -1, 1],
    [-1, 0, 2, -1, 1, -1],
    [1, -2, 0, 1, 1, -1],
    [0, 1, -1, 0, -1, 1],
    [1, -1, -1, 1, 0, 0],
    [-1, 1, 1, -1, 0, 0]
])

mutate(quiver1, 3)

array([[ 0.,  1., -1.,  0., -1.,  1.],
       [-1.,  0.,  1.,  1.,  0., -1.],
       [ 1., -1.,  0., -1.,  1.,  0.],
       [ 0., -1.,  1.,  0.,  1., -1.],
       [ 1.,  0., -1., -1.,  0.,  1.],
       [-1.,  1.,  0.,  1., -1.,  0.]])

After a mutation we are free to mutate again at another vertex. For example we can do
$$
\mu_2\mu_4\mu_3(Q)
$$
we define a function for applying sequences of mutations. Note that the above sequence would be $[3,4,2]$, since we do 3 first.

In [4]:
def mutation_sequence(quiver, sequence):
    current_quiver = quiver
    for vertex in sequence:
        current_quiver = mutate(current_quiver, vertex)
    return current_quiver

In [5]:
sequence = [3,4,2]
mutation_sequence(quiver1, sequence)

array([[ 0., -1.,  2., -3., -1.,  1.],
       [ 1.,  0., -1.,  1.,  0.,  0.],
       [-2.,  1., -0.,  1.,  1., -1.],
       [ 3., -1., -1.,  0., -1.,  1.],
       [ 1.,  0., -1.,  1.,  0.,  0.],
       [-1.,  0.,  1., -1.,  0.,  0.]])

# Mutation sequences

We are interested in finding all of the possible mutation sequences. One property of mutation is that $\mu_k^2=1$, so we can exclude mutation sequences which adjacent vertices being equal, for example $[4,2,4,3,3,1]$, since the two $\mu_3$ will cancel to give the sequence $[4,2,4,1]$. Apart from this we are free to choose any mutations we want.

In [6]:
def no_adjacent_duplicates(sequence):
    'Returns true if sequence has no adjacent duplicate entries. False otherwise'
    for idx in range(len(sequence)-1):
        if sequence[idx] == sequence[idx+1]:
            return False
    return True

def combinations_without_adjacent_duplicates(vertices, depth):
    'Returns all possible sequences without adjacent duplicate vertices of given length (here called depth)'
    all_sequences = itertools.product(vertices, repeat=depth)
    return [sequence for sequence in all_sequences if no_adjacent_duplicates(sequence)]

An example set of possible mutation sequences of length 2, with 4 possible choices of vertex. For example $(1,2)$ means we can do $\mu_2\mu_1(Q)$

In [7]:
combinations_without_adjacent_duplicates([1,2,3,4], 2)

[(1, 2),
 (1, 3),
 (1, 4),
 (2, 1),
 (2, 3),
 (2, 4),
 (3, 1),
 (3, 2),
 (3, 4),
 (4, 1),
 (4, 2),
 (4, 3)]

# Quivers obtained from all possible mutation sequences

We apply the mutation sequences found above to find all quivers that can be obtained by any mutation sequence (up to a given length) applied to an initial quiver 

In [8]:
def equivalent_quivers(quiver, depth):
    '''
    Returns a dictionary of quivers equivalent to the initial one. The dictionary values are the quivers and the keys
    are the mutation sequences. The initial key is just an empty pair of brackets (). The keys for quivers obtained by
    a single mutation are of the form (k,) [note the comma]. For longer sequences the keys are of the form (i, j, k) 
    [note the commas and blank spaces]
    '''
    pairs = {"()":quiver}
    vertices = range(quiver.shape[0])
    
    for i in tqdm(range(1, depth+1)):
        combs = combinations_without_adjacent_duplicates(vertices, i)
        for comb in combs:
            previous_quiver = pairs[repr(comb[:-1])]
            new_quiver = mutate(previous_quiver, comb[-1])
            pairs[repr(comb)] = new_quiver
    return pairs

We now have a dictionary with equivalent quivers. We can use the mutation sequences as keys to obtain the quivers. Note the strict key format

In [9]:
for i in range(3):
    start_time = timeit.default_timer()
    equivalent_quivers(quiver1, i)
    print(timeit.default_timer() - start_time)

0it [00:00, ?it/s]


0.00621630500000947


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 2366.99it/s]


0.003077653999980612


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 936.54it/s]

0.00432520200001818


In [10]:
equivalent_quivers(quiver1, 2)['()']

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 822.65it/s]


array([[ 0,  1, -1,  0, -1,  1],
       [-1,  0,  2, -1,  1, -1],
       [ 1, -2,  0,  1,  1, -1],
       [ 0,  1, -1,  0, -1,  1],
       [ 1, -1, -1,  1,  0,  0],
       [-1,  1,  1, -1,  0,  0]])

In [11]:
equivalent_quivers(quiver1, 2)['(0,)']

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 846.91it/s]


array([[ 0., -1.,  1.,  0.,  1., -1.],
       [ 1.,  0.,  1., -1.,  0., -1.],
       [-1., -1.,  0.,  1.,  1.,  0.],
       [ 0.,  1., -1.,  0., -1.,  1.],
       [-1.,  0., -1.,  1.,  0.,  1.],
       [ 1.,  1.,  0., -1., -1.,  0.]])

In [12]:
equivalent_quivers(quiver1, 2)['(0, 1)']

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 870.46it/s]


array([[ 0.,  1.,  1., -1.,  1., -2.],
       [-1., -0., -1.,  1., -0.,  1.],
       [-1.,  1.,  0.,  0.,  1., -1.],
       [ 1., -1.,  0.,  0., -1.,  1.],
       [-1., -0., -1.,  1.,  0.,  1.],
       [ 2., -1.,  1., -1., -1.,  0.]])

# Are two quivers equivalent?

Given a pair of quivers, we are interested in finding out if we can obtain one by mutation the other. Actually we don't really mind if we relabel the vertices on either quiver.

Obtaining a quiver by relabelling vertices is equivalent to conjugating its B-matrix by a permutation matrix. We write a function get_permutation that returns all possible relabelled B-matrices.

We then use this to write a function that will check to see if there is a mutation sequence (up to a given length) applied to a first quiver that will give a permuation of a second quiver. Finally we have a function to determine which permutation this is, though it could have been determined as part of the mutation_sequences_between_quivers function

In [13]:
def conjugate(mat1, mat2):
    return np.matmul(np.matmul(mat1, mat2), np.linalg.inv(mat1))

def get_permutations(quiver):
    'Returns a list of all matrices given by conjugating quiver with a permutation matrix'
    dim = quiver.shape[0]
    permutation_mats = [np.array(mat) for mat in list(itertools.permutations(np.eye(dim)))]
    return [conjugate(mat, quiver) for mat in permutation_mats]

def mutation_sequences_between_quivers(quiver1, quiver2, depth, early_stop=True):
    '''
    Return a list of all possible mutation sequences, up to length "depth", that can be apply to quiver1 to give a 
    relabelling of quiver2. If early_stop is true this will stop as soon as it finds a sequence, otherwise it
    will search for all possibilities
    '''
    results = []
    quiver2_permutations = get_permutations(quiver2)
    pairs = {"()":quiver1}
    vertices = range(quiver1.shape[0])
    
    for i in tqdm(range(1, depth+1)):
        combs = combinations_without_adjacent_duplicates(vertices, i)
        for comb in combs:
            previous_quiver = pairs[repr(comb[:-1])]
            new_quiver = mutate(previous_quiver, comb[-1])
            pairs[repr(comb)] = new_quiver
            if any((new_quiver == quiver).all() for quiver in quiver2_permutations):
                results.append(comb)
                if early_stop:
                    return results
    return results

def which_permutation(quiver1, quiver2):
    dim = quiver1.shape[0]
    permutation_mats = [np.array(mat) for mat in list(itertools.permutations(np.eye(dim)))]
    pairs = [[np.matmul(np.matmul(mat, quiver1), np.linalg.inv(mat)), mat] for mat in permutation_mats]
    return [pair[1] for pair in pairs if np.array_equal(pair[0], quiver2)]

An example, by default early_stop is set to false, we will just find the first sequence that works. This means that 
$$
\mu_2\mu_0(Q_2) = \sigma(Q_3)
$$
for a permutation $\sigma$ which is not given by the function. We identify this below.

In [14]:
quiver2 = np.array([
    [ 0., -1.,  0.,  0.],
    [ 1.,  0., -1.,  0.],
    [ 0.,  1.,  0.,  1.],
    [ 0.,  0., -1.,  0.]
])

quiver3 = np.array([
    [ 0., -0.,  1.,  0.],
    [-0., -0.,  1., -1.],
    [-1., -1.,  0.,  0.],
    [ 0.,  1.,  0.,  0.]
])

mutation_sequences_between_quivers(quiver2, quiver3, depth=3)

 33%|███████████████████████████▋                                                       | 1/3 [00:00<00:00, 1159.93it/s]


[(0, 2)]

If early_stop is set to false it will find all sequences that work, up to the given depth.

In [15]:
mutation_sequences_between_quivers(quiver2, quiver3, depth=3, early_stop=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 633.20it/s]


[(0, 2),
 (0, 3),
 (2, 0),
 (2, 1),
 (3, 0),
 (1, 0, 3),
 (1, 2, 1),
 (1, 3, 0),
 (3, 1, 0)]

Let's check that this actually works. We look at the first sequence

In [16]:
mutation_sequence(quiver2, [0, 2])

array([[ 0.,  1.,  0.,  0.],
       [-1.,  0.,  1.,  0.],
       [ 0., -1., -0., -1.],
       [-0.,  0.,  1.,  0.]])

The which_permuation function gives us the permuation that, applied to the first argument, gives the second. Here we can see that $Q_3$ and $\mu_2\mu_0(Q_1)$ are the same, up to permutation

In [17]:
sigma = which_permutation(quiver3, mutation_sequence(quiver2, [0, 2]))
sigma

[array([[0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.]])]

Finally we can see $\sigma(Q_3)$ and compare this with $\mu_2\mu_0(Q_1)$ above

In [18]:
conjugate(sigma, quiver3)

array([[[ 0.,  1.,  0.,  0.],
        [-1.,  0.,  1.,  0.],
        [ 0., -1.,  0., -1.],
        [ 0.,  0.,  1.,  0.]]])

# Periodic quivers

A quiver is said to be periodic if it is fixed by a mutation sequence (up to permutation). This fits in nicely with the functions we have defined above, we can simply use our mutation_sequences_between_quivers function, using the same quiver as both arguments.

We can see that $Q_2$ is both period 2 and 3, with mutation sequences as given below.

In [19]:
mutation_sequences_between_quivers(quiver2, quiver2, depth=3, early_stop=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 460.02it/s]


[(0, 1), (2, 3), (1, 0, 1), (3, 2, 3)]